In [5]:
import pickle
import re


In [6]:
def read_sentences(path):
    Sentences = []
    Sentence_Classes = []
    Temp_Sentence = []
    Temp_Sentence_Classes = []
    with open(path, 'r', encoding="utf-8") as file:
        # Iterate through each line
        for line in file:

            # Split the line by the tab character
            parts = line.split('\t')


            # Check if there are at least two parts
            if len(parts) >= 2:
                word = parts[0]
                class_ = parts[1].strip()  # strip() to remove leading/trailing whitespace

                # Do something with the word and class, for example, print them
                #print("Word:", word)
                #print("Class:", class_)


                Temp_Sentence.append(word)
                Temp_Sentence_Classes.append(class_)
                #print(Temp_Sentence)
            else:
                #print("Line does not contain a tab character")
                Sentences.append(Temp_Sentence)
                Sentence_Classes.append(Temp_Sentence_Classes)
                Temp_Sentence = []
                Temp_Sentence_Classes = []
    return Sentences, Sentence_Classes


In [21]:
path_normal_reading = r'C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\zuco1_normalreading_ner.txt'

path_task_reading = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\zuco1_taskreading_ner.txt"

path_sentiment = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\zuco1_sentiment_ner.txt"

normal_reading_sentences, normal_reading_classes = read_sentences(path_normal_reading)
task_reading_sentences, task_reading_classes = read_sentences(path_task_reading)
sentiment_sentences, sentiment_classes = read_sentences(path_sentiment)

#combine
Sentences_labels = [item for sublist in [normal_reading_sentences, task_reading_sentences, sentiment_sentences] for item in sublist]
Sentence_Classes = [item for sublist in [normal_reading_classes, task_reading_classes, sentiment_classes] for item in sublist]


In [22]:
print(Sentences_labels[0])

['Henry', 'Ford', 'advocated', 'long-time', 'associate', 'Harry', 'Bennett', 'to', 'take', 'the', 'spot', '.']


In [23]:
print(Sentence_Classes[0])

['B-PER', 'I-PER', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O']


In [24]:
def read_EEG_embeddings_labels(path):
    with open(path, 'rb') as file:
        EEG_word_level_embeddings = pickle.load(file)
        EEG_word_level_labels = pickle.load(file)
    return EEG_word_level_embeddings, EEG_word_level_labels

In [25]:
def get_sentences_labels(EEG_word_level_labels):
    sentences = []
    current_sentence = []
    # Iterate through the words
    for i in range(len(EEG_word_level_labels)):
        # Check if the word marks the start of a new sentence
        word = EEG_word_level_labels[i]
        if word == "SOS":
            # If it does, append the current sentence to the list of sentences
            if current_sentence:
                sentences.append(current_sentence)
            # Start a new sentence
            current_sentence = []
        else:
            # Add the word to the current sentence
            current_sentence.append(word)
    # Append the last sentence
    if current_sentence:
        sentences.append(current_sentence)
    return sentences

In [29]:
def combine_sentences(sentences):
    # Combine each internal list into a string
    combined_sentences = [' '.join(sentence) for sentence in sentences]
    # Initialize an empty list to store the list of words for each sentence
    list_of_words = []
    # Iterate over each sentence in the list
    for sentence in combined_sentences:
        # Split the sentence into a list of words including punctuation
        words = re.findall(r'\b\w+\b|[^\w\s]', sentence)
        # Append the list of words to the list of lists
        list_of_words.append(words)
    return list_of_words


In [41]:
def get_named_entities(Sentences_labels, Sentence_Classes):
    List_of_NE = []
    List_of_NE_Labels = []
    Named_Entity = []
    Named_Entity_Label = []

    for i in range(len(Sentences_labels)):
        current_sentence = Sentences_labels[i]
        current_sentence_label = Sentence_Classes[i]

        for j in range(len(current_sentence)):
            current_word = current_sentence[j]
            #print(current_word)
            current_word_label = current_sentence_label[j]
            #print(current_word_label)
            if current_word_label != 'O':
                Named_Entity.append(current_word)
                Named_Entity_Label.append(current_word_label)
            else:
                if Named_Entity:
                    List_of_NE.append(Named_Entity)
                    List_of_NE_Labels.append(Named_Entity_Label)

                    Named_Entity = []
                    Named_Entity_Label = []
    return List_of_NE, List_of_NE_Labels

In [45]:
def is_named_entity_in_sentences(named_entity, sentences):
    counter = 0
    for sentence in sentences:
        for i in range(len(sentence) - len(named_entity) + 1):
            if sentence[i:i+len(named_entity)] == named_entity:
                counter += 1

    if counter > 0:
        return True, counter
    else:
        return False, counter

In [26]:
train_path = r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs_Sentence.pkl"
test_path = r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\Test_EEG_Text_Pairs_Sentence.pkl"

EEG_word_level_embeddings, EEG_word_level_labels = read_EEG_embeddings_labels(train_path)
Test_EEG_word_level_embeddings, Test_EEG_word_level_labels = read_EEG_embeddings_labels(test_path)

In [27]:
EEG_word_level_sentences = get_sentences_labels(EEG_word_level_labels)
Test_EEG_word_level_sentences = get_sentences_labels(Test_EEG_word_level_labels)


In [30]:
# Combine the sentences
list_of_words = combine_sentences(EEG_word_level_sentences)
Test_list_of_words = combine_sentences(Test_EEG_word_level_sentences)

In [42]:
# Get the named entities
List_of_NE, List_of_NE_Labels = get_named_entities(Sentences_labels, Sentence_Classes)

In [47]:
# Check if each named entity is in the list of sentences
counter_sum = 0
for NE in List_of_NE:
    boolean, counter = is_named_entity_in_sentences(NE, Test_list_of_words)
    if boolean:
        print(f"Named entity {NE} found in {counter} sentences")
    else:
        print(f"Named entity {NE} found in {counter} sentences")
    counter_sum += counter

Named entity ['Henry', 'Ford'] found in 0 sentences
Named entity ['Harry', 'Bennett'] found in 0 sentences
Named entity ['Edsel'] found in 0 sentences
Named entity ['Eleanor'] found in 0 sentences
Named entity ['Edsel'] found in 0 sentences
Named entity ['Henry', 'Ford', 'II'] found in 0 sentences
Named entity ['Aldous', 'Leonard', 'Huxley'] found in 0 sentences
Named entity ['United', 'States'] found in 5 sentences
Named entity ['Huxley', 'family'] found in 0 sentences
Named entity ['Huxley'] found in 0 sentences
Named entity ['F.', 'Matthias', 'Alexander'] found in 0 sentences
Named entity ['Huxley'] found in 0 sentences
Named entity ['Godalming'] found in 0 sentences
Named entity ['Surrey'] found in 0 sentences
Named entity ['England'] found in 22 sentences
Named entity ['Leonard', 'Huxley'] found in 0 sentences
Named entity ['Julia', 'Arnold'] found in 0 sentences
Named entity ['Thomas', 'Henry', 'Huxley'] found in 0 sentences
Named entity ['Darwin'] found in 0 sentences
Named enti

157906